## Data Cleaning

In [57]:
import pandas as pd
import datetime
from datetime import datetime

In [4]:
# Load the 85d0f106b2e27c20.csv file
Option = pd.read_csv('/Users/baronma/Desktop/85d0f106b2e27c20.csv')

In [5]:
Option.head()

,date,symbol_flag,exdate,cp_flag,strike_price,best_offer,impl_volatility,optionid,ticker,index_flag,issuer,exercise_style
0,20130102,1,20130104,C,40000,20.4,NaN,100270777,AXP,0,AMERICAN EXPRESS CO,A
1,20130102,1,20130104,C,44000,15.4,NaN,100270778,AXP,0,AMERICAN EXPRESS CO,A
2,20130102,1,20130104,C,45000,14.4,NaN,100270779,AXP,0,AMERICAN EXPRESS CO,A
3,20130102,1,20130104,C,46000,13.4,NaN,100270780,AXP,0,AMERICAN EXPRESS CO,A
4,20130102,1,20130104,C,47000,12.4,NaN,100270781,AXP,0,AMERICAN EXPRESS CO,A


In [8]:
Option.drop(['symbol_flag', 'optionid', 'index_flag', 'issuer', 'exercise_style'], axis = 1, inplace = True)

In [9]:
# Load the 0fde65dd1062ea27.csv file
Underlying = pd.read_csv('/Users/baronma/Desktop/0fde65dd1062ea27.csv')

In [10]:
Underlying.head()

,secid,date,ticker,close
0,101375,20130102,AXP,58.75
1,101375,20130103,AXP,59.00
2,101375,20130104,AXP,59.61
3,101375,20130107,AXP,59.86
4,101375,20130108,AXP,60.20


In [63]:
Interest = pd.read_csv('/Users/baronma/Downloads/FRED-DTB3.csv')
Interest['Value'] = Interest['Value'].apply(lambda x: x/100)
Interest['Date'] = pd.to_datetime(Interest['Date'], format = '%Y-%m-%d').dt.strftime('%Y%m%d')
Interest.head()

,Date,Value
0,20190620,0.0209
1,20190619,0.0213
2,20190618,0.0217
3,20190617,0.0218
4,20190614,0.0215


In [11]:
# Get the original strike price
Option['strike_price'] = Option['strike_price'].apply(lambda x: x/1000)

In [73]:
# Left join AAPL_stock to AAPL_option in order to add the underlying stock price to the option
Options = pd.merge(Option, Underlying, on = ['date', 'ticker'], how = 'left')

In [74]:
Options.head()

,date,exdate,cp_flag,strike_price,best_offer,impl_volatility,ticker,secid,close
0,20130102,20130104,C,40.0,20.4,NaN,AXP,101375,58.75
1,20130102,20130104,C,44.0,15.4,NaN,AXP,101375,58.75
2,20130102,20130104,C,45.0,14.4,NaN,AXP,101375,58.75
3,20130102,20130104,C,46.0,13.4,NaN,AXP,101375,58.75
4,20130102,20130104,C,47.0,12.4,NaN,AXP,101375,58.75


In [76]:
Interest.rename(columns = {'Date' : 'date'}, inplace = True)
Interest['date'] = Interest['date'].astype(int)
Options_final = pd.merge(Options, Interest, on = ['date'], how = 'left')

In [77]:
Options_final.drop(['secid'], axis = 1, inplace = True)
Options_final.rename(columns = {'close' : 'underlying_price'}, inplace = True)
# Change the text date to the standard date format
Options_final['date'] = pd.to_datetime(Options_final['date'], format = '%Y%m%d')
Options_final['exdate'] = pd.to_datetime(Options_final['exdate'], format = '%Y%m%d')
# Calculate the maturity days
Options_final['maturity'] = (Options_final['exdate'] - Options_final['date']).dt.days

In [78]:
Options_withoutNaN = Options_final[Options_final['impl_volatility'].notnull()]
len(Options_withoutNaN)

1966689

In [79]:
# One hot encoding to the 'C=Call, P=Put' column
Options_df = pd.get_dummies(data=Options_withoutNaN, columns=['cp_flag'])

In [80]:
Options_df.head()

,date,exdate,strike_price,best_offer,impl_volatility,ticker,underlying_price,Value,maturity,cp_flag_C,cp_flag_P
12,2013-01-02,2013-01-04,60.0,0.03,0.181376,AXP,58.75,0.0008,2,1,0
13,2013-01-02,2013-01-04,62.5,0.05,0.450289,AXP,58.75,0.0008,2,1,0
14,2013-01-02,2013-01-04,65.0,0.05,0.676564,AXP,58.75,0.0008,2,1,0
15,2013-01-02,2013-01-04,67.5,0.50,1.369425,AXP,58.75,0.0008,2,1,0
16,2013-01-02,2013-01-04,70.0,0.01,0.888123,AXP,58.75,0.0008,2,1,0


In [81]:
Options_df.rename(columns = {'Value' : 'interest_rate'}, inplace = True)

In [83]:
Options_df = Options_df[['ticker', 'date', 'exdate', 'maturity', 'strike_price', 'best_offer', 'impl_volatility', 
                         'underlying_price', 'interest_rate', 'cp_flag_C', 'cp_flag_P']]

In [84]:
Options_df.head()

,ticker,date,exdate,maturity,strike_price,best_offer,impl_volatility,underlying_price,interest_rate,cp_flag_C,cp_flag_P
12,AXP,2013-01-02,2013-01-04,2,60.0,0.03,0.181376,58.75,0.0008,1,0
13,AXP,2013-01-02,2013-01-04,2,62.5,0.05,0.450289,58.75,0.0008,1,0
14,AXP,2013-01-02,2013-01-04,2,65.0,0.05,0.676564,58.75,0.0008,1,0
15,AXP,2013-01-02,2013-01-04,2,67.5,0.50,1.369425,58.75,0.0008,1,0
16,AXP,2013-01-02,2013-01-04,2,70.0,0.01,0.888123,58.75,0.0008,1,0


In [85]:
Options_df.to_csv('Options.csv', index = False)